In [ ]:
    def get_category_type(self, row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    def getDfFoursquareNearbyVenues(self, limit=500, radius=1000):
        lat = self.madrid_lat_long[0]
        lng = self.madrid_lat_long[1]
        url = 'https://api.foursquare.com/v2/venues/explore'
        params = dict(
            client_id='Q0PPOJTVQYJHGKB3VDY0FHAE221XVEYZEKI2AY4BSGXXR2IM',
            client_secret='JBVYOWC44RZQDVCWWIXVSYULYY4PRGWESQBBH54SKHO2XI4L',
            v='20200405',
            ll='%s,%s' % (lat, lng),
            radius='%s' % (radius),
            limit=limit)
        resp = requests.get(url=url, params=params)
        data = json.loads(resp.text)
        venues = data['response']['groups'][0]['items']
        nearby_venues = pd.json_normalize(venues)  # flatten JSON
        print('Found %s nearby venues at %s,%s' % (len(nearby_venues.index), lat, lng))

        # filter columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        if (len(nearby_venues.index) > 0):
            nearby_venues = nearby_venues.loc[:, filtered_columns]
            # filter the category for each row
            nearby_venues['venue.categories'] = nearby_venues.apply(self.get_category_type, axis=1)
            # clean columns
            nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        return nearby_venues

    def venue_suggestion(self):
        # venues near the restaurant lat-long defined:
        df_venues = self.getDfFoursquareNearbyVenues()
        venue_options = ['Plaza', 'Art Museum', 'Monument / Landmark', 'Art Gallery', 'Church', 'Palace', 'Opera House',
                         'Historic Site', 'Theater', 'Movie Theater', 'Indie Movie Theater', 'Garden']
        self.df_venue_options = df_venues[df_venues['categories'].isin(venue_options)].reset_index()
        if len(self.df_venue_options) > 2:
            return messagebox.showinfo('Suggestion:',
                                       f"Nearby you can visit a {self.df_venue_options['categories'][0].upper()} called {self.df_venue_options['name'][0]} "
                                       f"and a {self.df_venue_options['categories'][1].upper()} called {self.df_venue_options['name'][1]}!")
        elif len(self.df_venue_options) > 1:
            return messagebox.showinfo('Suggestion:',
                                       f"Nearby you can visit a {self.df_venue_options['categories'][0].upper()} called {self.df_venue_options['name'][0]}!")
        else:
            return messagebox.showinfo('Suggestion:', 'There is no tourist attraction available.')